In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.ei import EnsembleIntegration
import eipy.utils
from eipy.additional_ensembles import MeanAggregation, CES
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import datasets

In [2]:
# If data is multi-class, run a check on the allowable base and meta models.

base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(multi_class="multinomial"),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}

In [3]:

"""
For filtering base predictors by whether or not they rely on heursitics for multiclass extension

natively_multi_class_predictors = ["XGBClassifier",
"BernoulliNB",
"DecisionTreeClassifier",
"ExtraTreeClassifier",
"GaussianNB",
"KNeighborsClassifier",
"LabelPropagation",
"LabelSpreading",
"LinearDiscriminantAnalysis",
"LinearSVC", #(setting multi_class=”crammer_singer”)
"LogisticRegression", #(setting multi_class=”multinomial”)
"LogisticRegressionCV", #(setting multi_class=”multinomial”)
"MLPClassifier",
"NearestCentroid",
"QuadraticDiscriminantAnalysis",
"RadiusNeighborsClassifier",
"RandomForestClassifier",
"RidgeClassifier",
"RidgeClassifierCV"]

base_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}
"""

'\nFor filtering base predictors by whether or not they rely on heursitics for multiclass extension\n\nnatively_multi_class_predictors = ["XGBClassifier",\n"BernoulliNB",\n"DecisionTreeClassifier",\n"ExtraTreeClassifier",\n"GaussianNB",\n"KNeighborsClassifier",\n"LabelPropagation",\n"LabelSpreading",\n"LinearDiscriminantAnalysis",\n"LinearSVC", #(setting multi_class=”crammer_singer”)\n"LogisticRegression", #(setting multi_class=”multinomial”)\n"LogisticRegressionCV", #(setting multi_class=”multinomial”)\n"MLPClassifier",\n"NearestCentroid",\n"QuadraticDiscriminantAnalysis",\n"RadiusNeighborsClassifier",\n"RandomForestClassifier",\n"RidgeClassifier",\n"RidgeClassifierCV"]\n\nbase_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}\n'

In [4]:
"""https://dev.pages.lis-lab.fr/scikit-multimodallearn/tutorial/auto_examples/combo/plot_combo_3_views_3_classes.html#
multi modal multi-class toy data generation"""

def generate_data(n_samples, lim):
    """Generate random data in a rectangle"""
    lim = np.array(lim)
    n_features = lim.shape[0]
    data = np.random.random((n_samples, n_features))
    data = (lim[:, 1]-lim[:, 0]) * data + lim[:, 0]
    return data
seed = 12
np.random.seed(seed)

n_samples = 300

modality_0 = np.concatenate((generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]])))

modality_1 = np.concatenate((generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]])))

modality_2 = np.concatenate((generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]])))

y = np.zeros(3*n_samples, dtype=np.int64)
y[n_samples:2*n_samples] = 1
y[2*n_samples:] = 2


In [5]:
X_0_train, X_0_test, y_train, y_test = train_test_split(modality_0, y, test_size=0.2, random_state=3, stratify=y)
X_1_train, X_1_test, _,_ = train_test_split(modality_1, y, test_size=0.2, random_state=3, stratify=y)
X_2_train, X_2_test, _,_ = train_test_split(modality_2, y, test_size=0.2, random_state=3, stratify=y)

In [6]:
data_train = {
                "Modality_0": X_0_train,
                "Modality_1": X_1_train,
                "Modality_2": X_2_train
                }

data_test = {
                "Modality_0": X_0_test,
                "Modality_1": X_1_test,
                "Modality_2": X_2_test
                }

In [7]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        n_jobs=-1,
                        random_state=42,
                        project_name="toy",
                        model_building=True,
                        )



In [8]:
for name, modality in data_train.items():
    EI.train_base(modality, y_train, modality_name=name)

Training base predictors on Modality_0...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%


modality       Modality_0                               
base predictor       ADAB DT GB KNN LR MLP NB RF SVM XGB
sample                  0  0  0   0  0   0  0  0   0   0
0                       1  1  1   1  1   1  1  1   1   1
1                       0  2  0   0  2   0  0  2   0   2
2                       0  0  0   0  0   0  0  0   0   0
3                       0  0  0   0  2   2  2  0   2   0
4                       0  2  2   0  0   0  0  0   0   2
..                    ... .. ..  .. ..  .. .. ..  ..  ..
139                     0  0  0   2  0   0  0  0   0   0
140                     0  0  0   0  0   0  0  0   0   0
141                     1  1  1   1  2   2  2  1   2   1
142                     1  1  1   1  1   1  1  1   1   1
143                     1  1  1   1  1   1  1  1   1   1

[144 rows x 10 columns]
modality       Modality_0                               
base predictor       ADAB DT GB KNN LR MLP NB RF SVM XGB
sample                  0  0  0   0  0   0  0  0   0   0
0     

KeyError: 'labels'

In [ ]:
EI.meta_training_data[0]

In [ ]:
pd.set_option('display.max_columns', None)
EI.meta_test_data[0]

In [ ]:
EI.base_summary

In [ ]:
EI.train_meta(meta_predictors=base_predictors)

In [ ]:
EI.meta_summary["metrics"]

In [ ]:
y_pred = EI.predict(X_dict=data_test, meta_model_key="XGB")
y_pred = np.round(y_pred)
y_pred

In [ ]:
y_test

In [ ]:
accuracy = sum([1*(y==y_hat)+0*(y!=y_hat) for y,y_hat in list(zip(y_test, y_pred))])/len(y_test)
accuracy # =179/180

In [ ]:
EI.meta_summary['thresholds']["XGB"]

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

Modality_a = X[:, 0:2]
Modality_b = X[:, 2:4]

X_a_train, X_a_test, y_train, y_test = train_test_split(Modality_a, y, test_size=0.2, random_state=3, stratify=y)
X_b_train, X_b_test, _,_ = train_test_split(Modality_b, y, test_size=0.2, random_state=3, stratify=y)

In [ ]:
iris_data_train = {
                "Modality_a": X_a_train,
                "Modality_b": X_b_train
                }

iris_data_test = {
                "Modality_a": X_a_test,
                "Modality_b": X_b_test
                }

In [ ]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}

In [ ]:
EI_iris = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        n_jobs=-1,
                        random_state=0,
                        project_name="iris",
                        model_building=True,
                        )


In [ ]:
for name, modality in iris_data_train.items():
    EI_iris.train_base(modality, y_train, modality_name=name)

In [ ]:
EI_iris.meta_training_data

In [ ]:
EI_iris.train_meta(meta_predictors=base_predictors)

In [ ]:
EI_iris.meta_summary["metrics"]

In [ ]:
y_pred_iris = EI_iris.predict(X_dict=iris_data_test, meta_model_key="SVM")
y_pred_iris = np.round(y_pred_iris)
y_pred_iris

In [ ]:
accuracy = sum([1*(y==y_hat)+0*(y!=y_hat) for y,y_hat in list(zip(y_test, y_pred_iris))])/len(y_test)
accuracy

In [ ]:
import pandas as pd
import numpy as np

# Create a dictionary with 2D arrays as values
data = {
    'Column1': [np.array([1, 2]), np.array([3, 4]), np.array([5, 6])],
    'Column2': [np.array([10, 20]), np.array([30, 40]), np.array([50, 60])]
}

# Create a DataFrame with MultiIndex
df = pd.DataFrame()

# Iterate over the dictionary items and reshape 2D arrays into sub-columns
for key, value in data.items():
    num_cols = value[0].shape[0]  # Get the shape of the 2D array
    print(num_cols)
    for i in range(num_cols):
        # Create sub-columns for each element of the 2D array
        sub_column_name = f'SubColumn{i + 1}'
        df[(key, sub_column_name)] = [row[i] for row in value]

# Print the DataFrame
print(df)
